# 배터리 데이터 사이클러 분류 시스템

이 노트북은 배터리 충방전 데이터 경로를 자동으로 분석하여 **토요(Toyo)** 또는 **PNE** 사이클러로 분류합니다.

**분류 기준**:
- **PNE**: Pattern 폴더가 존재하는 경로
- **Toyo**: Pattern 폴더가 없는 경로

In [ ]:
# 필수 라이브러리 import
import os
import re
from pathlib import Path
from typing import List, Dict, Optional, Tuple
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

## 유틸리티 함수

In [ ]:
def check_cycler(path: str) -> str:
    """
    경로의 사이클러 타입을 식별합니다.
    
    Parameters:
    -----------
    path : str
        확인할 데이터 경로
    
    Returns:
    --------
    str
        'PNE' 또는 'Toyo' 또는 'Unknown'
    """
    if not os.path.exists(path):
        return "Unknown (경로 없음)"
    
    # Pattern 폴더 존재 여부로 PNE와 Toyo 구분
    pattern_path = os.path.join(path, "Pattern")
    
    if os.path.isdir(pattern_path):
        return "PNE"
    else:
        return "Toyo"


def extract_capacity_from_path(path: str) -> Optional[float]:
    """
    경로명에서 용량 정보를 추출합니다.
    
    Parameters:
    -----------
    path : str
        분석할 경로
    
    Returns:
    --------
    float or None
        추출된 용량(mAh), 없으면 None
    """
    # 정규 표현식으로 mAh 추출
    match = re.search(r'(\d+([\-.]\d+)?)mAh', path)
    if match:
        capacity_str = match.group(1).replace('-', '.')
        return float(capacity_str)
    return None


def get_directory_info(path: str) -> Dict[str, any]:
    """
    디렉토리의 메타 정보를 추출합니다.
    
    Parameters:
    -----------
    path : str
        분석할 디렉토리 경로
    
    Returns:
    --------
    dict
        디렉토리 정보 (하위 폴더 수, 파일 수 등)
    """
    info = {
        'exists': False,
        'subdirs': 0,
        'files': 0,
        'has_pattern_folder': False,
        'file_types': set()
    }
    
    if not os.path.exists(path):
        return info
    
    info['exists'] = True
    
    try:
        items = os.listdir(path)
        
        for item in items:
            item_path = os.path.join(path, item)
            
            if os.path.isdir(item_path):
                info['subdirs'] += 1
                if item.lower() == 'pattern':
                    info['has_pattern_folder'] = True
            else:
                info['files'] += 1
                # 파일 확장자 추출
                _, ext = os.path.splitext(item)
                if ext:
                    info['file_types'].add(ext.lower())
    
    except PermissionError:
        info['error'] = '권한 없음'
    except Exception as e:
        info['error'] = str(e)
    
    return info


def classify_battery_paths(path_list: List[str]) -> pd.DataFrame:
    """
    여러 배터리 데이터 경로를 일괄 분류합니다.
    
    Parameters:
    -----------
    path_list : List[str]
        분류할 경로 리스트
    
    Returns:
    --------
    pd.DataFrame
        분류 결과가 담긴 데이터프레임
    """
    results = []
    
    for path in path_list:
        # 기본 정보
        cycler_type = check_cycler(path)
        dir_info = get_directory_info(path)
        capacity = extract_capacity_from_path(path)
        
        # 경로의 마지막 부분만 추출 (가독성)
        path_name = os.path.basename(path)
        
        results.append({
            '경로': path,
            '폴더명': path_name,
            '사이클러': cycler_type,
            '용량(mAh)': capacity if capacity else '-',
            '하위폴더수': dir_info['subdirs'],
            '파일수': dir_info['files'],
            'Pattern폴더': '있음' if dir_info['has_pattern_folder'] else '없음',
            '존재여부': '존재' if dir_info['exists'] else '없음'
        })
    
    df = pd.DataFrame(results)
    return df

## 경로 입력

분석할 배터리 데이터 경로를 입력합니다. 아래 방법 중 하나를 선택하세요.

In [ ]:
# 방법 1: 직접 리스트로 입력 (기본값: 기존 경로들)
paths = [
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\250219_250319_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 101-200cyc",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\250304_250404_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 201-300cyc",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\250317_251231_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 301-400cyc",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\A1_MP1_4500mAh_T23_1",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\A1_MP1_4500mAh_T23_2",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\A1_MP1_4500mAh_T23_3",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\Dateset_A1_Gen4 2C ATL MP2 [45V 4470mAh] [23] blk2",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\Gen4 2C ATL MP2 [45V 4470mAh] [23] blk7 - 240131",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\M1 ATL [45V 4175mAh]",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\Q7M Inner ATL_45V 1689mAh BLK1 20EA [23] - 250304",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\Q7M Main ATL [45V_1680mAh][23] blk7 20ea - 250228",
    r"C:\Users\Ryu\Python_project\data\원본 - 복사본\Rawdata\Q7M Sub ATL [45v 2068mAh] [23] - 250219r"
]

print(f"총 {len(paths)}개 경로가 입력되었습니다.")

### (선택) 방법 2: 새 경로 추가

필요시 아래 셀의 주석을 해제하고 새로운 경로를 추가할 수 있습니다.

In [ ]:
# # 새 경로 추가 예제
# new_path = r"C:\Users\Ryu\새로운\경로\예시"
# paths.append(new_path)
# print(f"새 경로 추가됨: {new_path}")

## 사이클러 자동 분류

입력된 모든 경로를 자동으로 분류합니다.

In [ ]:
# 경로 분류 실행
results_df = classify_battery_paths(paths)

# 결과 출력
print("\n=== 배터리 데이터 사이클러 분류 결과 ===")
print(f"\n총 분석 경로: {len(results_df)}개")
print(f"\n사이클러 타입별 통계:")
print(results_df['사이클러'].value_counts())

# 전체 결과 테이블 (경로 제외 - 가독성)
display_df = results_df.drop(columns=['경로'])
display(display_df)

## 상세 분석

사이클러 타입별로 데이터를 그룹화하여 분석합니다.

In [ ]:
# 토요 사이클러 데이터
toyo_df = results_df[results_df['사이클러'] == 'Toyo']
print(f"\n=== 토요(Toyo) 사이클러 - {len(toyo_df)}개 ===")
if len(toyo_df) > 0:
    display(toyo_df[['폴더명', '용량(mAh)', '하위폴더수', '파일수']])
else:
    print("토요 사이클러 데이터가 없습니다.")

# PNE 사이클러 데이터
pne_df = results_df[results_df['사이클러'] == 'PNE']
print(f"\n=== PNE 사이클러 - {len(pne_df)}개 ===")
if len(pne_df) > 0:
    display(pne_df[['폴더명', '용량(mAh)', '하위폴더수', '파일수']])
else:
    print("PNE 사이클러 데이터가 없습니다.")

## 시각화

분류 결과를 그래프로 시각화합니다.

In [ ]:
import matplotlib.pyplot as plt

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 사이클러 타입별 개수
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 파이 차트: 사이클러 타입 분포
cycler_counts = results_df['사이클러'].value_counts()
axes[0].pie(cycler_counts.values, labels=cycler_counts.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('사이클러 타입 분포', fontsize=14, fontweight='bold')

# 바 차트: 용량별 분포
capacity_data = results_df[results_df['용량(mAh)'] != '-'].copy()
if len(capacity_data) > 0:
    capacity_data['용량(mAh)'] = capacity_data['용량(mAh)'].astype(float)
    capacity_grouped = capacity_data.groupby('사이클러')['용량(mAh)'].mean()
    capacity_grouped.plot(kind='bar', ax=axes[1], color=['#FF6B6B', '#4ECDC4'])
    axes[1].set_title('사이클러별 평균 용량', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('평균 용량 (mAh)', fontsize=12)
    axes[1].set_xlabel('사이클러 타입', fontsize=12)
    axes[1].tick_params(axis='x', rotation=0)
else:
    axes[1].text(0.5, 0.5, '용량 정보 없음', ha='center', va='center', fontsize=14)
    axes[1].set_title('사이클러별 평균 용량', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 결과 저장

분류 결과를 CSV 파일로 저장합니다.

In [ ]:
# 결과를 CSV로 저장
output_file = 'battery_classification_results.csv'
results_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n결과가 '{output_file}'에 저장되었습니다.")

# 파일 절대 경로 출력
abs_path = os.path.abspath(output_file)
print(f"저장 위치: {abs_path}")

## 요약 및 다음 단계

### 완료된 작업
✅ 배터리 데이터 경로 자동 분류 (Toyo/PNE)
✅ 디렉토리 메타 정보 추출
✅ 용량 정보 자동 추출
✅ 분류 결과 시각화
✅ CSV 파일로 결과 저장

### 다음 단계 (선택)
이 노트북을 기반으로 다음과 같은 기능을 추가할 수 있습니다:

1. **데이터 로딩**: 사이클러 타입에 따라 실제 데이터 파일 로딩
2. **데이터 분석**: 충방전 사이클 데이터 분석 및 시각화
3. **모듈화**: 자주 사용하는 함수를 별도 `.py` 파일로 분리
4. **자동화**: 새로운 데이터가 추가될 때 자동으로 분류

### 사용법
- 새로운 경로를 분류하려면: "경로 입력" 셀에서 `paths` 리스트를 수정
- 재분류하려면: "사이클러 자동 분류" 셀부터 다시 실행